# Combination Generator

In [ ]:
import pandas as pd
import numpy as np
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0")
db = lusas.database()

Read the excel definitions

In [ ]:
file_name = "Datafiles/12 Combinations.xlsx"
actions_sheet = "Actions"
is610 = False    # Create combination 6.10 or 6.10a and 6.10b

# Action Definitions

In [ ]:
# Permanent Actions
df_permanent_actions = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(0,4), header=1).dropna()

# Variable Actions
df_variable_actions = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(5,10), header=1).dropna()

# Prestress Actions
df_prestress_actions = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(11,14), header=1).dropna()
df_prestress_actions.columns = df_prestress_actions.columns.str.replace(r'.\d+', '', regex=True)

# Accidental Actions
df_accidental_actions = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(15,17), header=1).dropna()

# Seismic Actions
df_seismic_actions = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(18,20), header=1).dropna()

# Loadcase Definitions

In [ ]:
# Permanent Loadcases in LUSAS 
df_perm_loadcases = pd.read_excel(file_name, sheet_name="PermanentLoadcases")
# Mapping of Action Name to list of corresponding LUSAS Loadcases
dict_perm_loadcases = df_perm_loadcases.set_index('Action Name').T.to_dict('list')

# Variable Loadcases in LUSAS 
df_var_loadcases = pd.read_excel(file_name, sheet_name="VariableLoadcases")
# Mapping of Action Name to list of corresponding LUSAS Loadcases
dict_var_loadcases = df_var_loadcases.set_index('Action Name').T.to_dict('list')

# Accidental Loadcases in LUSAS 
df_acc_loadcases = pd.read_excel(file_name, sheet_name="AccidentalLoadcases")
# Mapping of Action Name to list of corresponding LUSAS Loadcases
dict_acc_loadcases = df_acc_loadcases.set_index('Action Name').T.to_dict('list')

# Seismic Loadcases in LUSAS 
df_seismic_loadcases = pd.read_excel(file_name, sheet_name="SeismicLoadcases")
# Mapping of Action Name to list of corresponding LUSAS Loadcases
dict_seismic_loadcases = df_seismic_loadcases.set_index('Action Name').T.to_dict('list')

In [ ]:
# Debug
#print(df_acc_loadcases.head())
#print(dict_acc_loadcases)

## Helper functions

In [ ]:
# Helper method to add different loadcase types to a smart combination
def add_loadset_to_smart_comb(smart_comb, loadcase:str, beneficial:float, adverse:float) -> bool:
    if db.existsLoadset(loadcase):
        loadset = db.getLoadset(loadcase)
        type_code = loadset.getTypeCode()

        if type_code <= 2:
            smart_comb.addEntry(beneficial, (adverse-beneficial), loadset)
        elif type_code == 3:
            envelope1 = win32.CastTo(loadset, "IFEnvelope")
            envelope2 = envelope1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope2)
        elif type_code == 6:
            smart1 = win32.CastTo(loadset, "IFSmartCombination")
            smart2 = smart1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart2)
        return True
    else:
        return False


In [ ]:
# Helper function to add permanent actions to the smart combination
def add_permanent_actions_to_smart_combination(smart_comb, include_factors=True, include_reduction=False):
    for action_type in dict_perm_loadcases.keys():
        # Factors for this type
        if include_factors:
            beneficial = df_permanent_actions[df_permanent_actions['Perm Action Name'] == action_type]['Beneficial']
            adverse    = df_permanent_actions[df_permanent_actions['Perm Action Name'] == action_type]['Adverse']
            reduction  = df_permanent_actions[df_permanent_actions['Perm Action Name'] == action_type]['Reduction']
            reduction  = reduction if include_reduction else 1.0
        else:
            beneficial = 1.0
            adverse    = 1.0
            reduction  = 1.0

        # LUSAS Loadcases
        for loadcase in dict_perm_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, beneficial, adverse * reduction):
                print(f"Permanent loadcase '{loadcase}' does not exists")

In [ ]:
# Helper function to add factored prestress loads to the smart combination
def add_prestress_loads_to_smart_combination(smart_comb, include_factors=True):
    for ip, rowp in df_prestress_actions.iterrows():
        loadcase = rowp['Loadcase Name']
        if not isinstance(loadcase, str) and np.isnan(loadcase) : continue        
        
        # Factors for this type
        if include_factors:
            beneficial = rowp['Beneficial']
            adverse    = rowp['Adverse']
        else:
            beneficial = 1.0
            adverse    = 1.0

        if not add_loadset_to_smart_comb(smart_comb, loadcase, beneficial, adverse):
            print(f"Prestress loadcase '{loadcase}' does not exists")

## Equ 6.10

$$ \sum_{j\ge1}{\gamma_{G,j}G_{k,j} + \gamma_pP + \gamma_{Q,1}Q_{k,1}}  + \sum_{i\gt1}{ \gamma_{Q,i}\psi_{0,i}Q_{k,i}}

In [ ]:
def add_loadset_to_smart_comb(smart_comb, loadcase:str, beneficial:float, adverse:float) -> bool:
    if db.existsLoadset(loadcase):
        loadset = db.getLoadset(loadcase)
        type_code = loadset.getTypeCode()

        if type_code <= 2:
            smart_comb.addEntry(beneficial, (adverse-beneficial), loadset)
        elif type_code == 3:
            envelope1 = win32.CastTo(loadset, "IFEnvelope")
            envelope2 = envelope1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope2)
        elif type_code == 6:
            smart1 = win32.CastTo(loadset, "IFSmartCombination")
            smart2 = smart1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart2)
        return True
    else:
        return False


Equ 6.10

In [ ]:
if is610:

    uls_envelope = db.createEnvelope("ULS 6.10")
    uls_envelope.setTreeLocation("Design Envelopes", True)

    # Consider each variable action as a leading action
    for i, row in df_variable_actions.iterrows():
        action = row["Var Action Name"]
        load_factor = row["DesignFactor"]
        comb_factor = row["CombinationFactor"]
        
        smart_comb = db.createCombinationSmart(f"ULS - 610 - {action}")

        # Permanent actions
        add_permanent_actions_to_smart_combination(smart_comb)

        # Prestress actions
        add_prestress_loads_to_smart_combination(smart_comb)

        # Variable actions
        for action_type in dict_var_loadcases.keys():
            # Determine variable factor
            if action_type == action:
                adverse = load_factor                        # Leading Variable
            else:
                adverse = load_factor * comb_factor          # Accompanying Variable

            # LUSAS Loadcases
            for loadcase in dict_var_loadcases[action_type]:
                if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

                if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                    print(f"Variable loadcase '{loadcase}' does not exists")


        # Add combination to overall design envelope
        uls_envelope.addEntry(smart_comb)
        uls_envelope.addEntry(smart_comb.getAssocLoadset())
        smart_comb.setTreeParent(uls_envelope)
        smart_comb.getAssocLoadset().setTreeParent(uls_envelope.getAssocLoadset())


## 6.10a Or 6.10b

In [ ]:
if not is610:
    uls_envelope = db.createEnvelope("ULS 6.10a or 6.10b")
    uls_envelope.setTreeLocation("Design Envelopes", True)

    uls_envelope_610a = db.createEnvelope("ULS 6.10a")
    uls_envelope_610b = db.createEnvelope("ULS 6.10b")

    uls_envelope.addEntry(uls_envelope_610a)
    uls_envelope.addEntry(uls_envelope_610a.getAssocLoadset())

    uls_envelope.addEntry(uls_envelope_610b)
    uls_envelope.addEntry(uls_envelope_610b.getAssocLoadset())

    uls_envelope_610a.setTreeParent(uls_envelope)
    uls_envelope_610a.getAssocLoadset().setTreeParent(uls_envelope.getAssocLoadset())
        
    uls_envelope_610b.setTreeParent(uls_envelope)
    uls_envelope_610b.getAssocLoadset().setTreeParent(uls_envelope.getAssocLoadset())


# 6.10a

$$ \sum_{j\ge1}{\gamma_{G,j}G_{k,j} + \gamma_pP + \gamma_{Q,1}\psi_{0,i}Q_{k,1}}  + \sum_{i\gt1}{ \gamma_{Q,i}\psi_{0,i}Q_{k,i}}

In [ ]:
if not is610:
    # Expression 610a
    # All variable actions consider a combination factor
    smart_comb = db.createCombinationSmart(f"ULS - 610a")

    # Permanent actions
    add_permanent_actions_to_smart_combination(smart_comb)

    # Prestress actions
    add_prestress_loads_to_smart_combination(smart_comb)

    # Variable actions
    for i, row in df_variable_actions.iterrows():
        action_type = row["Var Action Name"]
        load_factor = row["DesignFactor"]
        comb_factor = row["CombinationFactor"]

        # Add all of the LUSAS Loadcases of the action type
        for loadcase in dict_var_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, load_factor * comb_factor):
                print(f"Variable loadcase '{loadcase}' does not exists")


    # Add combination to 6.10a envelope
    uls_envelope_610a.addEntry(smart_comb)
    uls_envelope_610a.addEntry(smart_comb.getAssocLoadset())
    smart_comb.setTreeParent(uls_envelope_610a)
    smart_comb.getAssocLoadset().setTreeParent(uls_envelope_610a.getAssocLoadset())

# 6.10b

$$ \sum_{j\ge1}{\xi_j\gamma_{G,j}G_{k,j} + \gamma_pP + \gamma_{Q,1}Q_{k,1}}  + \sum_{i\gt1}{ \gamma_{Q,i}\psi_{0,i}Q_{k,i}}

In [ ]:
if not is610:
    # Expression 610b
    # Consider each variable action as a leading action
    for i, row in df_variable_actions.iterrows():
        action = row["Var Action Name"]
        load_factor = row["DesignFactor"]
        comb_factor = row["CombinationFactor"]
        
        smart_comb = db.createCombinationSmart(f"ULS - 6.10b - {action}")

        # Permanent actions with associated reduction factor
        add_permanent_actions_to_smart_combination(smart_comb, include_reduction=True)

        # Prestress actions
        add_prestress_loads_to_smart_combination(smart_comb)

        # Variable actions
        for action_type in dict_var_loadcases.keys():
            # Determine variable factor
            if action_type == action:
                adverse = load_factor                        # Leading Variable
            else:
                adverse = load_factor * comb_factor          # Accompanying Variable

            # Add all of the LUSAS Loadcases of the action type
            for loadcase in dict_var_loadcases[action_type]:
                if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

                if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                    print(f"Variable loadcase '{loadcase}' does not exists")


        # Add leading variable action combination to 6.10b envelope
        uls_envelope_610b.addEntry(smart_comb)
        uls_envelope_610b.addEntry(smart_comb.getAssocLoadset())
        smart_comb.setTreeParent(uls_envelope_610b.getAssocLoadset())
        smart_comb.getAssocLoadset().setTreeParent(uls_envelope_610b)

# Accidental 6.11

$$ \sum_{j\ge1}{G_{k,j} + P + A_{d} + (\psi_{1,1} | \psi_{2,1})Q_{k,1} + \sum_{i\gt1}{ \psi_{2,i}Q_{k,i}}}

In [ ]:
accidental_envelope = db.createEnvelope("Accidental 6.11")
accidental_envelope.setTreeLocation("Design Envelopes", True)

# Accidental scenarios
for i1, rows in df_accidental_actions.iterrows():
    acc_action = rows['Acc Action Name']
    if not isinstance(acc_action, str) and np.isnan(acc_action) : continue 
    
    # Consider each variable action as a leading action
    for i, row in df_variable_actions.iterrows():
        var_action  = row["Var Action Name"]
        load_factor = 1.0
        freq_factor = row["FrequentFactor"]
        quasi_factor = row["QuasiFactor"]
        
        smart_comb = db.createCombinationSmart(f"Accidental - 611 - {acc_action} - {var_action}")

        # Permanent actions
        add_permanent_actions_to_smart_combination(smart_comb, include_factors=False)

        # Prestress actions
        add_prestress_loads_to_smart_combination(smart_comb, include_factors=False)

        # Accidental Action
        # Factors for this type, should be 1
        factor = rows['AccFactor']
        # Accidental loadcases for the current accidental action
        for loadcase_name in dict_acc_loadcases[acc_action]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue
            
            if not add_loadset_to_smart_comb(smart_comb, loadcase_name, 0.0, factor):
                print(f"Accidental loadcase '{loadcase_name}' does not exists")


        # Variable actions
        for action_type in dict_var_loadcases.keys():
            # Determine variable factor
            if action_type == action:
                adverse = load_factor                        # Leading Variable
            else:
                adverse = load_factor * freq_factor          # Accompanying Variable

            # LUSAS Loadcases
            for loadcase in dict_var_loadcases[action_type]:
                if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

                if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                    print(f"Variable loadcase '{loadcase}' does not exists")


        # Add combination to envelope
        accidental_envelope.addEntry(smart_comb)
        accidental_envelope.addEntry(smart_comb.getAssocLoadset())
        smart_comb.setTreeParent(accidental_envelope)
        smart_comb.getAssocLoadset().setTreeParent(accidental_envelope.getAssocLoadset())    

# Seismic 6.12

$$ \sum_{j\ge1}{G_{k,j} + P + A_{Ed} + \sum_{i\gt1}{ \psi_{2,i}Q_{k,i}}}

In [ ]:
seismic_envelope = db.createEnvelope("Seismic 6.12")
seismic_envelope.setTreeLocation("Design Envelopes", True)

# Seismic scenarios
for i1, rows in df_seismic_actions.iterrows():
    action_name = rows['Seismic Action Name']
    if not isinstance(action_name, str) and np.isnan(action_name) : continue    

    smart_comb = db.createCombinationSmart(f"Seismic - 612 - {action_name}")

    # Permanent actions
    add_permanent_actions_to_smart_combination(smart_comb, include_factors=False)

    # Prestress actions
    add_prestress_loads_to_smart_combination(smart_comb, include_factors=False)

    # Seimic Action
    # Factors for this type, should be 1
    factor = rows['SeismicFactor']
    # Seismic loadcase should be a single envelope
    for loadcase_name in dict_seismic_loadcases.keys():
        if not add_loadset_to_smart_comb(smart_comb, loadcase_name, 0.0, factor):
            print(f"Seismic loadcase '{loadcase_name}' does not exists")


    # All variable actions with quasi combination factors
    for i, row in df_variable_actions.iterrows():
        action_type = row["Var Action Name"]
        quasi_factor = row["QuasiFactor"]

        # Add all of the LUSAS Loadcases of the action type
        for loadcase in dict_var_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, quasi_factor):
                print(f"Variable loadcase '{loadcase}' does not exists")


    # Add combination to envelope
    seismic_envelope.addEntry(smart_comb)
    seismic_envelope.addEntry(smart_comb.getAssocLoadset())
    smart_comb.setTreeParent(seismic_envelope)
    smart_comb.getAssocLoadset().setTreeParent(seismic_envelope.getAssocLoadset())

# Characteristic 6.14

$$ \sum_{j\ge1}{G_{k,j} + P + Q_{k,1}}  + \sum_{i\gt1}{ \psi_{0,i}Q_{k,i}}

In [ ]:
characteristic_envelope = db.createEnvelope("SLS Characteristic 6.14")
characteristic_envelope.setTreeLocation("Design Envelopes", True)

# Consider each variable action as a leading action
for i, row in df_variable_actions.iterrows():
    action = row["Var Action Name"]
    load_factor = 1.0
    comb_factor = row["CombinationFactor"]
    
    smart_comb = db.createCombinationSmart(f"SLS Char - 614 - {action}")

    # Permanent actions
    add_permanent_actions_to_smart_combination(smart_comb, include_factors=False)

    # Prestress actions
    add_prestress_loads_to_smart_combination(smart_comb, include_factors=False)

    # Variable actions
    for action_type in dict_var_loadcases.keys():
        # Determine variable factor
        if action_type == action:
            adverse = load_factor                        # Leading Variable
        else:
            adverse = load_factor * comb_factor          # Accompanying Variable

        # LUSAS Loadcases
        for loadcase in dict_var_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                print(f"Variable loadcase '{loadcase}' does not exists")


    # Add combination to overall design envelope
    characteristic_envelope.addEntry(smart_comb)
    characteristic_envelope.addEntry(smart_comb.getAssocLoadset())
    smart_comb.setTreeParent(characteristic_envelope)
    smart_comb.getAssocLoadset().setTreeParent(characteristic_envelope.getAssocLoadset())

# Frequent 6.15

$$ \sum_{j\ge1}{G_{k,j} + P + \psi_{1,1}Q_{k,1}}  + \sum_{i\gt1}{ \psi_{2,i}Q_{k,i}}

In [ ]:
frequent_envelope = db.createEnvelope("SLS Frequent 6.15")
frequent_envelope.setTreeLocation("Design Envelopes", True)

# Consider each variable action as a leading action
for i, row in df_variable_actions.iterrows():
    action = row["Var Action Name"]
    load_factor = 1.0
    freq_factor = row["FrequentFactor"]
    quasi_factor = row["QuasiFactor"]   
    
    smart_comb = db.createCombinationSmart(f"SLS Freq - 615 - {action}")

    # Permanent actions
    add_permanent_actions_to_smart_combination(smart_comb, include_factors=False)

    # Prestress actions
    add_prestress_loads_to_smart_combination(smart_comb, include_factors=False)

    # Variable actions
    for action_type in dict_var_loadcases.keys():
        # Determine variable factor
        if action_type == action:
            adverse = load_factor * freq_factor           # Leading Variable
        else:
            adverse = load_factor * quasi_factor          # Accompanying Variable

        # LUSAS Loadcases
        for loadcase in dict_var_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                print(f"Variable loadcase '{loadcase}' does not exists")


    # Add combination to overall design envelope
    frequent_envelope.addEntry(smart_comb)
    frequent_envelope.addEntry(smart_comb.getAssocLoadset())
    smart_comb.setTreeParent(frequent_envelope)
    smart_comb.getAssocLoadset().setTreeParent(frequent_envelope.getAssocLoadset())

# Quasi-permanent 6.16

$$ \sum_{j\ge1}{G_{k,j} + P + \sum_{i\gt1}{ \psi_{2,i}Q_{k,i}}}

In [ ]:
quasi_envelope = db.createEnvelope("SLS Quasi-Permanent 6.16")
quasi_envelope.setTreeLocation("Design Envelopes", True)    

# All variable actions consider a quasi-permanent factor
smart_comb = db.createCombinationSmart(f"SLS Quasi - 616")

# Permanent actions
add_permanent_actions_to_smart_combination(smart_comb, include_factors=False)

# Prestress actions
add_prestress_loads_to_smart_combination(smart_comb, include_factors=False)

# Variable actions
for i, row in df_variable_actions.iterrows():
    action_type = row["Var Action Name"]
    quasi_factor = row["QuasiFactor"]

    # Add all of the LUSAS Loadcases of the action type
    for loadcase in dict_var_loadcases[action_type]:
        if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

        if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, quasi_factor):
            print(f"Variable loadcase '{loadcase}' does not exists")


# Add combination to envelope
quasi_envelope.addEntry(smart_comb)
quasi_envelope.addEntry(smart_comb.getAssocLoadset())
smart_comb.setTreeParent(quasi_envelope)
smart_comb.getAssocLoadset().setTreeParent(quasi_envelope.getAssocLoadset())